# Crypto Lab 2: Putting it all together

**Prerequisites:**
- Complete all parts 1-5 of this lab
- Complemte the IoT lab

Now the time has come to put everything together. Your task is to create your own secret messaging scheme for the Micro:Bit.

- Part 1: Verify that the radio works. I.e., two microbits can communicate.
- Part 2: Using ChaCha20, we begin securing the communication between two Micro:Bits. We use a static key to begin with and provide code for the receiver-side. 
- Part 3: Extend with MAC functionality. We provide a custom hashing algortihm for the MAC.  
- Part 4: Extend so that the key is no longer static, but computed using DHKE. 



---

> **NB: Make sure you have completed Crypto Lab 1: Basic introductions before starting on this lab**

In this lab, you’ll take what you learned in the previous crypto lab and bring it to life on real hardware using the **Micro:Bit**. You’ll also be introduced to a new algorithm: **ChaCha20**. 

To follow along, you’ll need two Micro:Bits - ask your TA's if you only have one. 

## 1 Verify that two Micro:Bits can communicate using the Radio

First, let’s make sure the two Micro:Bits can communicate with each other!  
Copy the code provided in [lab2-1.py](/crypto-lab/lab-2/lab2-1.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3) and upload it to **both** of your devices.  

Next, choose one Micro:Bit to act as the **sender** and the other as the **receiver**.  
- On the sender, press the buttons to transmit a message.  
- On the receiver, check that the message is displayed correctly.  

> ⚠️ **Important:** Before you run the program, update the `GROUP_NUMBER` in the code to match your assigned rat group.  
> Your group number is created by combining your team number with A/B (where A = 0 and B = 1).  

Examples:  
- Team **1A** → `10`  
- Team **1B** → `11` 

#### Question 1.1: Explain briefly what happens in the code snippet (maximum 3 lines of text)

#### Question 1.2: Right now, the messages are sent “in the clear” over the radio - what does this mean?

#### Question 1.3: What happens if another team nearby sets the same group number?

Before moving on, make sure the setup works:  
Press **A** or **B** on the sender, and check that the corresponding message shows up on the receiver’s display.  


---

## 2 Encryption and decryption of messages

Now that you’ve confirmed the two Micro:Bits can communicate, it’s time to take the next step: implementing a simple **encryption and decryption** mechanism.  

1. Copy the code from [lab2-2.py](/crypto-lab/lab-2/lab2-2.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3).  
2. Send the code to both Micro:Bits.  
3. ⚠️ Don’t forget to update the `GROUP_NUMBER` here as well, just like in Task 1.  

If you look at the new code, you’ll notice that most of the previous functions remain unchanged.  
However, a few **new elements have been added**!

If you scroll down to the bottom of the Python file, you’ll find a **ChaCha20 module**.  
Don’t worry if it looks complex, you are **not expected to fully understand it** in this course - however it will become relevant in later cryptographic courses.

As seen in the preparation material, ChaCha20 is a widely used stream cipher.

#### Question 2.1: Why does ChaCha20 only have an encrypt function, not decrypt? 

You’ll also notice some **new constants** in the code:  

- `GLOBAL_KEY` → a static global key written in binary format  
- `MESSAGE_1` / `MESSAGE_2` → example messages you can use to test the functionality  

💡 **Why this matters:** In real-world cryptography, securely sharing and managing keys is one of the hardest challenges.  

For now, we use a fixed key for simplicity, but later you’ll see how key exchange and management become critical for secure communication.  

In [9]:
GLOBAL_KEY = b'chacha20!'
MESSAGE_1 = "Hello World"
MESSAGE_2 = "Goodbye World"

Furthermore, both functions:`def send_mode()` and `def receive_mode()` are altered. They both call a new function:`on_receive()` before displaying the data: 

In [ ]:
def on_receive(received_bytes):
    try: 
        decrypted_bytes = chacha20_encrypt(
            received_bytes, GLOBAL_KEY)
        data = decrypted_bytes.decode('utf-8')
        return data
    except TypeError:
        return "TYPE ERROR"
    except:
        return "WEIRD ERROR"

#### Question 2.2: Explain shortly what happens in the `on_receive` function.

Additionally, `def send_mode()` has been altered to send the messages written as constants instead of "A" or "B" as in part 1. Here, a new function `on_send()` is used instead of `radio.send()`.

Using the `on_receive()` function, let's try to fill out `on_send()`!   
Follow the steps as written in the TODO:

In [ ]:
def on_send(msg):
    """
    TODO: 
    Implement a send function that encodes the inputted
    message to bytes, encrypts the bytes using chacha20_encrypt()
    and then sends the encrypted bytes over the radio.
    """
    pass # Remove this line and implement your solution here

<details>
<summary><strong>💡 Hint</strong></summary>

Try using the same logic as in on_receive() to fill out on_send(). NOEN BEDRE HINT HER???
</details>

If your code works as intended, you should now be able to send encrypted messages. Verify that it works through ????

## 3 Extend with MAC technology



Great job! Now that the simple encryption through ChaCha20 is in place, let's further implement MAC (Message Authentication Code):

1. Copy the code from [lab2-3.py](/crypto-lab/lab-2/lab2-3.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3).  
2. Send the code to both Micro:Bits.  
3. ⚠️ Don’t forget to update the `GROUP_NUMBER` here as well, just like in Task 1 and 2. 
4. ⚠️ Make sure to copy the code from task 2 into the function `on_send()`.

Similarily as in the last task, some parts of the code is now altered!

Firstly, let's look at the changes made in `on_receive()`: 

In [ ]:
def on_receive(received_bytes):
    try:
        mac, msg = split_data(received_bytes)
        decrypted_bytes = chacha20_encrypt(
            msg, GLOBAL_KEY)
        data = decrypted_bytes.decode('utf-8')

        if verify_mac(GLOBAL_KEY.decode('utf-8'),
                  data,
                  mac):
            return data
        else:
            return "INVALID MAC"
    except TypeError:
        return "TYPE ERROR"
    except:
        return "UNEXPECTED ERROR"
    
def split_data(data):
    mac = data[0:2]
    msg = data[2:]
    return mac, msg

The `on_receive()` function has been updated to include **verification of the MAC** sent over the radio.  

Here’s what happens step by step:  
1. The incoming data is split into two parts using the `split_data()` function:  
   - the fixed-length **MAC**  
   - the variable-length **message**  
2. The message is then **decrypted** and converted back into a readable string.  
3. Finally, the `verify_mac()` function checks whether the MAC matches the message.  
   - If it does, the message should be returned.  
   - If not, an error is shown (e.g., `"INVALID MAC"`).  

However, this `verify_mac()` function is not yet created. Neither is `generate_mac()`! A simple hash function is created to help you implement these functions. 

#### Question 3.1: In this lab, we created `simple_hash()` ourselves. Why is writing your own cryptographic functions usually a bad idea?

In [1]:
# Fill in your answer here.

Unfortunately, due to limitations we will have to use the simple_hash() in this case.

#### Task: Implement a MAC using `simple_hash()`  
Using the provided `simple_hash()` function, implement message authentication code functionality:

- **`generate_mac(key, message)`** — return a MAC (bytes) for the given key and message.  
- **`verify_mac(key, message, mac)`** — return `True` if `mac` is valid for the given key and message, otherwise `False`.

Your MAC should be **prepended** to the encrypted bytes in `on_send()` and **checked** in `on_receive()` before accepting a message.


<details>
<summary><strong>💡 Hint for generate_mac()</strong></summary>

1. Think about how you can **combine the key and the message** into a single string.  
2. Pass that combined string into the `simple_hash()` function.  
3. Return the resulting bytes as the MAC. 
</details>

<details>
<summary><strong>💡 Hint for verify_mac()</strong></summary>

1. Recreate the MAC yourself by calling `generate_mac(key, message)`.  
2. Compare the recreated MAC with the one received.  
3. Return `True` if they match, otherwise `False`.  
</details>